In [ ]:
import numpy as np 
import pandas as pd 
import io 
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import re
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
# GET the data
## Mounting path dari Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# GET the data
## Memuat Data
import pandas as pd

dataset = pd.read_csv("/content/drive/MyDrive/DataSet/output.csv")

In [ ]:
dataset['Sentimen'] = dataset['Sentimen'].replace(-1,0)
dataset

,Tweet,Sentimen
0,bius habis rt dahlan iskan gak jls capres dahl...,0
1,presiden prabowo presiden indonesia,1
2,capres prabowo bergitu bodoh ngecap uang ngak ...,0
3,kalo capres arb berani berfikir berani mimpi b...,1
4,najisorg gila doang yg dukung jdi presiden jok...,0
...,...,...
1880,kabur tanggung kemas calon capres jokowi,1
1881,berani pmrntah m g da yg d arepin klo dh ganti...,1
1882,anak medan dukung capres jk md ckp putar,1
1883,jelek komposisi udah bagus prabowo sbg preside...,1


In [ ]:
## Pra Pengolahan - Splitting
from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(
    dataset['Tweet'], dataset['Sentimen'], test_size=0.2,stratify=dataset['Sentimen'], random_state=20)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow import keras
from transformers import AutoTokenizer, TFAutoModel
import IPython

bert_tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p2")
def tokenisasi(teks):
    encode_dict = bert_tokenizer(teks,
                                   add_special_tokens = True,
                                   max_length = 128, 
                                   padding = 'max_length',
                                   truncation = True,
                                   return_attention_mask = True,
                                   return_tensors = 'tf',)

    tokenID = encode_dict['input_ids']
    attention_mask = encode_dict['attention_mask']

    return tokenID, attention_mask

def create_input(data):
    tokenID, input_mask = [], []
    for teks in data:
        token, mask = tokenisasi(teks)
        tokenID.append(token)
        input_mask.append(mask)
    
    return [np.asarray(tokenID, dtype=np.int32).reshape(-1, 128), 
            np.asarray(input_mask, dtype=np.int32).reshape(-1, 128)]

bert_model = TFAutoModel.from_pretrained("indobenchmark/indobert-base-p2", trainable=False)

def bert(hp):
    
    # Input layer
    input_token = keras.layers.Input(shape=(128,), dtype=np.int32,
                                        name="input_token")
    input_mask = keras.layers.Input(shape=(128,), dtype=np.int32,
                                   name="input_mask")

    # Embedding
    bert_embedding = bert_model([input_token, input_mask])[0]

    # Hidden layer
    num_hidden_units = hp.Int('num_hidden_units', min_value=32, max_value=512, step=16)
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.01)
    hidden = keras.layers.Dense(num_hidden_units, activation='relu',
                                kernel_regularizer=keras.regularizers.l2(hp.Choice('kernel_dense', values=[0.01, 0.001])))(bert_embedding)
    hidden = keras.layers.Dropout(dropout_rate)(hidden)

    # Output layer
    output = keras.layers.Dense(1, activation='sigmoid')(hidden)
    
    model = keras.models.Model(inputs=[input_token, input_mask], outputs=output)

    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 5e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
   
    return model

class ClearTrainingOutput(keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Some layers from the model checkpoint at indobenchmark/indobert-base-p2 were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at indobenchmark/indobert-base-p2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from keras_tuner.tuners import BayesianOptimization

bert_train_data = create_input(train_data)
bert_test_data = create_input(test_data)

tuner = BayesianOptimization(bert,
                             objective = 'val_accuracy', 
                             max_trials = 10,
                             directory = '/content/Hasil',
                             project_name = 'Sentiment-BERT',
                             overwrite = True)

tuner.search(bert_train_data, train_labels,
             batch_size=10, epochs=100,
             validation_data=(bert_test_data, test_labels),
             callbacks=[early_stop, ClearTrainingOutput()])

# Mendapatkan model terbaik
model = tuner.get_best_models()[0]

Trial 10 Complete [00h 08m 35s]
val_accuracy: 0.790927529335022

Best val_accuracy So Far: 0.8538005352020264
Total elapsed time: 01h 44m 15s


In [ ]:
## Evaluasi Model

test_loss, test_acc = model.evaluate(bert_test_data, test_labels)
print('Test accuracy:', test_acc)

12/12 [==============================] - 9s 267ms/step - loss: 0.4748 - accuracy: 0.8538
Test accuracy: 0.8538005352020264


In [ ]:
## Penyimpanan dan Memuat Kembali Model
import transformers
model.save('Data/model_mlp_sentiment.h5')
model = keras.models.load_model('Data/model_mlp_sentiment.h5', custom_objects={"TFBertModel": transformers.TFBertModel})